In [1]:
# ===== CELL 1: MOUNT DRIVE & PATHS =====
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os

# Base path for your data
base_path = '/content/drive/MyDrive/svnit_shared_task/shared_task/bhasha-workshop/Task1_I/Data'
kfold_folder = os.path.join(base_path, 'K_Fold')

# HuggingFace models
bert_models = {
    'bert-base-uncased': 'bert-base-uncased',
    'MurIL': 'google/muril-base-cased',
    'IndicBERT': 'ai4bharat/indic-bert'
}

# Traditional ML classifiers
ml_classifiers = {
    'LinearSVC': 'LinearSVC',
    'RandomForest': 'RandomForest',
    'LogisticRegression': 'LogisticRegression',
    'BernoulliNB': 'BernoulliNB'
}

print("Drive mounted and paths set.")
print("Base Path:", base_path)
print("K-Fold Path:", kfold_folder)


Mounted at /content/drive
Drive mounted and paths set.
Base Path: /content/drive/MyDrive/svnit_shared_task/shared_task/bhasha-workshop/Task1_I/Data
K-Fold Path: /content/drive/MyDrive/svnit_shared_task/shared_task/bhasha-workshop/Task1_I/Data/K_Fold


In [2]:
# ===== CELL 2: IMPORTS, FUNCTIONS, TRAIN & COMPARE =====
import pickle
import pandas as pd
import torch
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, TextClassificationPipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

languages = ["Bangla", "Hindi", "Malayalam", "Tamil", "Telugu"]
n_splits = 5
subset_size = 20  # small subset for faster test

# ===== HELPER FUNCTIONS =====
def load_all_folds(lang, subset=None):
    X_train_list, y_train_list, X_test_list, y_test_list = [], [], [], []
    for fold_num in range(1, n_splits+1):
        fold_path = os.path.join(kfold_folder, lang, f'fold_{fold_num}')
        with open(os.path.join(fold_path, 'X_train_raw.pkl'), 'rb') as f: X_train_list.extend(pickle.load(f))
        with open(os.path.join(fold_path, 'y_train.pkl'), 'rb') as f: y_train_list.extend(pickle.load(f))
        with open(os.path.join(fold_path, 'X_test_raw.pkl'), 'rb') as f: X_test_list.extend(pickle.load(f))
        with open(os.path.join(fold_path, 'y_test.pkl'), 'rb') as f: y_test_list.extend(pickle.load(f))
    train_df = pd.DataFrame({'text': X_train_list, 'label': y_train_list})
    test_df = pd.DataFrame({'text': X_test_list, 'label': y_test_list})
    if subset:
        train_df = train_df.iloc[:subset]
        test_df = test_df.iloc[:subset]
    return train_df, test_df

def tokenize_data(data, tokenizer):
    return tokenizer(data['text'], truncation=True, padding=True, max_length=128)

def compute_metrics(preds, labels):
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, zero_division=0)
    rec = recall_score(labels, preds, zero_division=0)
    f1 = f1_score(labels, preds, zero_division=0)
    return acc, prec, rec, f1

results = {}

# ===== HUGGINGFACE MODELS =====
for model_name, hf_name in bert_models.items():
    print(f"\n=================== MODEL: {model_name} ===================")
    tokenizer = AutoTokenizer.from_pretrained(hf_name, max_length=128)

    for lang in languages:
        train_df, test_df = load_all_folds(lang, subset=subset_size)
        train_dataset = Dataset.from_pandas(train_df)
        test_dataset = Dataset.from_pandas(test_df)
        train_tokenized = train_dataset.map(lambda x: tokenize_data(x, tokenizer), batched=True)
        test_tokenized = test_dataset.map(lambda x: tokenize_data(x, tokenizer), batched=True)

        model = AutoModelForSequenceClassification.from_pretrained(hf_name, num_labels=2).to(device)
        training_args = TrainingArguments(
            output_dir="/tmp/model_test",
            overwrite_output_dir=True,
            learning_rate=2e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=1,
            weight_decay=0.01,
            logging_steps=10,
            report_to=[]
        )
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_tokenized,
            eval_dataset=test_tokenized,
            tokenizer=tokenizer
        )
        trainer.train()

        pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=0 if device=='cuda' else -1)
        predictions = pipe(test_df['text'].tolist())
        pred_labels = [int(pred['label'].split('_')[1]) for pred in predictions]

        acc, prec, rec, f1 = compute_metrics(pred_labels, test_df['label'].tolist())
        print(f"{lang}: Acc={acc:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, F1={f1:.4f}")
        results.setdefault(lang, {})[model_name] = {'Acc': acc, 'Prec': prec, 'Rec': rec, 'F1': f1}

# ===== TRADITIONAL ML CLASSIFIERS =====
from sklearn.feature_extraction.text import CountVectorizer

for clf_name, clf_type in ml_classifiers.items():
    print(f"\n=================== ML CLASSIFIER: {clf_name} ===================")
    for lang in languages:
        train_df, test_df = load_all_folds(lang)
        X_train_raw, X_test_raw = train_df['text'].fillna(''), test_df['text'].fillna('')
        y_train, y_test = train_df['label'], test_df['label']

        # Using word-level n-grams
        vectorizer = CountVectorizer(ngram_range=(1,2))
        X_train = vectorizer.fit_transform(X_train_raw)
        X_test = vectorizer.transform(X_test_raw)

        if clf_type == 'LinearSVC': clf = LinearSVC()
        elif clf_type == 'RandomForest': clf = RandomForestClassifier()
        elif clf_type == 'LogisticRegression': clf = LogisticRegression(max_iter=500)
        elif clf_type == 'BernoulliNB': clf = BernoulliNB()

        if len(np.unique(y_train)) < 2:
            y_pred = np.full_like(y_test, fill_value=y_train.iloc[0])
        else:
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)

        acc, prec, rec, f1 = compute_metrics(y_pred, y_test)
        print(f"{lang}: Acc={acc:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, F1={f1:.4f}")
        results.setdefault(lang, {})[clf_name] = {'Acc': acc, 'Prec': prec, 'Rec': rec, 'F1': f1}

# ===== SUMMARY =====
print("\n\n===== MODEL COMPARISON SUMMARY =====")
for lang, metrics in results.items():
    print(f"\n--- {lang} ---")
    best_model = max(metrics.items(), key=lambda x: x[1]['F1'])
    for model_name, vals in metrics.items():
        print(f"{model_name}: Acc={vals['Acc']:.4f}, F1={vals['F1']:.4f}")
    print(f"Best model for {lang}: {best_model[0]} with F1={best_model[1]['F1']:.4f}")


Using device: cpu

=================== MODEL: bert-base-uncased ===================


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Step,Training Loss


Device set to use cpu


Telugu: Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000

=================== ML CLASSIFIER: LinearSVC ===================


/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Bangla: Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000
Hindi: Acc=0.9983, Prec=0.9982, Rec=1.0000, F1=0.9991
Malayalam: Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000
Tamil: Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000
Telugu: Acc=0.9950, Prec=0.9946, Rec=1.0000, F1=0.9973

=================== ML CLASSIFIER: RandomForest ===================
Bangla: Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000
Hindi: Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000
Malayalam: Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000
Tamil: Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000
Telugu: Acc=0.9967, Prec=0.9964, Rec=1.0000, F1=0.9982

=================== ML CLASSIFIER: LogisticRegression ===================
Bangla: Acc=0.9967, Prec=0.9953, Rec=1.0000, F1=0.9976
Hindi: Acc=0.9967, Prec=0.9963, Rec=1.0000, F1=0.9982
Malayalam: Acc=0.9967, Prec=0.9967, Rec=1.0000, F1=0.9983
Tamil: Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000
Telugu: Acc=0.9833, Prec=0.9822, Rec=1.0000, F1=0.9910

=================== ML CLAS

In [ ]:
# =================== RESULTS & OBSERVATIONS ===================

# Using device: cpu

# =================== MODEL: bert-base-uncased ===================
# Bangla:    Acc=0.7000, Prec=0.7000, Rec=1.0000, F1=0.8235
# Hindi:     Acc=0.4000, Prec=0.5714, Rec=0.5714, F1=0.5714
# Malayalam: Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000
# Tamil:     Acc=0.9500, Prec=1.0000, Rec=0.9500, F1=0.9744
# Telugu:    Acc=0.8500, Prec=1.0000, Rec=0.8500, F1=0.9189

# =================== MODEL: MurIL ===================
# Bangla:    Acc=0.6000, Prec=0.6667, Rec=0.8571, F1=0.7500
# Hindi:     Acc=0.7000, Prec=0.7000, Rec=1.0000, F1=0.8235
# Malayalam: Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000
# Tamil:     Acc=0.0000, Prec=0.0000, Rec=0.0000, F1=0.0000
# Telugu:    Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000

# =================== MODEL: IndicBERT ===================
# Bangla:    Acc=0.7000, Prec=0.7500, Rec=0.8571, F1=0.8000
# Hindi:     Acc=0.7500, Prec=0.7647, Rec=0.9286, F1=0.8387
# Malayalam: Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000
# Tamil:     Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000
# Telugu:    Acc=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000

# =================== ML CLASSIFIER: LinearSVC ===================
# Bangla:    Acc=1.0000, F1=1.0000
# Hindi:     Acc=0.9983, F1=0.9991
# Malayalam: Acc=1.0000, F1=1.0000
# Tamil:     Acc=1.0000, F1=1.0000
# Telugu:    Acc=0.9950, F1=0.9973

# =================== ML CLASSIFIER: RandomForest ===================
# Bangla:    Acc=1.0000, F1=1.0000
# Hindi:     Acc=1.0000, F1=1.0000
# Malayalam: Acc=1.0000, F1=1.0000
# Tamil:     Acc=1.0000, F1=1.0000
# Telugu:    Acc=0.9967, F1=0.9982

# =================== ML CLASSIFIER: LogisticRegression ===================
# Bangla:    Acc=0.9967, F1=0.9976
# Hindi:     Acc=0.9967, F1=0.9982
# Malayalam: Acc=0.9967, F1=0.9983
# Tamil:     Acc=1.0000, F1=1.0000
# Telugu:    Acc=0.9833, F1=0.9910

# =================== ML CLASSIFIER: BernoulliNB ===================
# Bangla:    Acc=0.9649, F1=0.9756
# Hindi:     Acc=0.9716, F1=0.9846
# Malayalam: Acc=0.9933, F1=0.9967
# Tamil:     Acc=1.0000, F1=1.0000
# Telugu:    Acc=0.9533, F1=0.9752

# =================== MODEL COMPARISON SUMMARY ===================
# Bangla   → Best: LinearSVC (F1=1.0000)
# Hindi    → Best: RandomForest (F1=1.0000)
# Malayalam→ Best: bert-base-uncased (F1=1.0000)
# Tamil    → Best: IndicBERT (F1=1.0000)
# Telugu   → Best: MurIL (F1=1.0000)
